### CNN classifier for microscopy images

Step 1 image processing



Different options are included. 

1. Downsample
2. gridcrop 
3. cell crop

Filter images first
1. Quality control
2. Filter by neuron count

In [4]:
# filtering

# read in quality control 

In [2]:
# basic processing Downsample


Step 2 create the model and train the classifier

In [3]:
# create model

# create_model_original.py
#
# Used to create the original model architecture

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

def create_model():
    model = Sequential()

    model.add(Conv2D(16, kernel_size=3, activation='relu', input_shape=(64, 64, 3)))
    model.add(Conv2D(8, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(8, kernel_size=3, activation='relu'))
    model.add(Conv2D(8, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(8, kernel_size=3, activation='relu'))
    model.add(Conv2D(8, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    return model


model = create_model()
print(model.summary())

# Save the entire model as a SavedModel.
# model.save('models/model_original')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# train_model.py
#
# Used to train models with data generator
#
# train_path - Paths for preprocessed images
# model_start - Path for model to begin training with 'model_original' (SavedModel format)
# model_name - Path to save newly trained model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

train_set_name = 'XCL_NPC_2_3'
model_start_name = 'model_original'
model_name = 'model_XCL_NPC_2_3'
steps_per_epoch = 64
max_epochs = 5000
early_stopping_patience = 200


###

train_path = 'data/' + train_set_name + '/train_set'
model_start_path = 'models/' + model_start_name
model_path = 'models/' + model_name

# Create data generators
datagen = ImageDataGenerator(rescale=1.0 / 255.0, validation_split=0.2)
train_gen = datagen.flow_from_directory(train_path, class_mode='categorical', batch_size=64, target_size=(64, 64), shuffle=True)
val_gen = datagen.flow_from_directory(train_path, class_mode='categorical', batch_size=64, target_size=(64, 64),  subset='validation')


# Load saved model and display the model's architecture
model = load_model(model_start_path)
print(model.summary())


# Pregenerate folder for end model
model.save(model_path)


# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=early_stopping_patience),
             ModelCheckpoint(filepath='models/' + model_path + '/best_weights.h5', monitor='val_loss', save_best_only=True)]


# Model Training
history = model.fit_generator(
        train_gen,
        steps_per_epoch=steps_per_epoch,
        epochs=max_epochs,
        callbacks=callbacks,  # Early stopping
        validation_data=val_gen,
        shuffle=True
        )

# Save the current model after training
model.save(model_path)

# List all data in history
print(history.history.keys())

# Create plots for accuracy and loss during training and save in model folder
# Plot accuracy history and save it into model directory
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(model_path + '/training_accuracy.png')
plt.clf()

# Plot loss history and save it into model directory
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(model_path + '/training_loss.png')



